<a href="https://colab.research.google.com/github/nafis-neehal/llm_projects/blob/main/Chandler-Bot/Chandler_Bot_DataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'friends-netflix-script-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1848343%2F3017794%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240204%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240204T225845Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9c1c57a030280e623e8bd7c76fa20fb627be6ed8f9effdf70582ae6226ae9ecf29450199fb2b097426f5b880b4db467543e6e07d34a67e7450baccf9c7d530e158d413d6bd5b4dac01dc8ad4de4b52cae7928a20dcf8076a54ad5db78eacb1eca82330d5bf7d7b5fbe496e2939ddb7dd1bab133be350b6e3bbf278eb208078c9a8f45b50acf57cef580e9820232f024e7f9ffbfa13b7afab12433e39f2682590622fed6d493f2df427f2f4b4f64b773d49403da9666ac113a484a5557daee11a2779f33d8ce93a262cd2f0e6294692f8c4867682a3c1a56b9e4ef872b21e41a891056dc3d9511599412e9bb8e16b4ed5683c6eaa92883071232abff1b610c204'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/friends-netflix-script-data/Friends.csv


In [ ]:
data = pd.read_csv("/kaggle/input/friends-netflix-script-data/Friends.csv")

# **Preprocesing**

1. Convered all speaker names to lower case
2. Replaced NaN speaker names with empty string ""

In [ ]:
data.Speaker = data.Speaker.str.lower()
data = data.fillna('')

In [ ]:
data

,Text,Speaker,Episode,Season,Show
0,Originally written by Marta Kauffman and David...,,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
1,Transcribed by guineapig.,,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,scene 1,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
3,There's nothing to tell! He's just some guy I...,monica,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
4,"C'mon, you're going out with the guy! There's...",joey,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
...,...,...,...,...,...
69969,Then I'm happy too. (They're still hugging - ...,ross,Episode-15-The One Where Estelle Dies,Season-10,Friends
69970,COMMERCIAL BREAK,,Episode-15-The One Where Estelle Dies,Season-10,Friends
69971,Estelle's memorial service. Joey is giving a ...,[scene,Episode-15-The One Where Estelle Dies,Season-10,Friends
69972,Thank you all for coming. We're here today to...,joey,Episode-15-The One Where Estelle Dies,Season-10,Friends


In [ ]:
chandler_speeches = data[data.Speaker.str.contains("chandler")]
chandler_idx = data[data.Speaker.str.contains("chandler")].index
prompt_idx = chandler_idx - 1
prompt_speeches = data.iloc[prompt_idx]

In [ ]:
data_formatted = pd.DataFrame()

In [ ]:
for i in range(prompt_speeches.shape[0]):
    instruction = "Below is a dialog between popular FRIENDS tv series character Chandler and his friends/coactors. Here, some other character says someethin which is used as the prompt and then Chandler responses. Learn how Chandler responses to everyone."
    string = instruction + "### Prompt:" + prompt_speeches.Text.iloc[i] + "### Chandler:" + chandler_speeches.Text.iloc[i]
    new_data = pd.DataFrame([{"text":string}])
    data_formatted = pd.concat([data_formatted, new_data], ignore_index=True)


In [ ]:
data_formatted

,text
0,Below is a dialog between popular FRIENDS tv s...
1,Below is a dialog between popular FRIENDS tv s...
2,Below is a dialog between popular FRIENDS tv s...
3,Below is a dialog between popular FRIENDS tv s...
4,Below is a dialog between popular FRIENDS tv s...
...,...
8162,Below is a dialog between popular FRIENDS tv s...
8163,Below is a dialog between popular FRIENDS tv s...
8164,Below is a dialog between popular FRIENDS tv s...
8165,Below is a dialog between popular FRIENDS tv s...


In [ ]:
data_formatted.to_csv("/kaggle/working/Chandler.csv")